In [17]:
!pip install opencv-python
!pip install dlib
!apt-get install -y cmake
!pip install numpy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


installing the dependencies

In [2]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2


--2025-01-09 20:37:52--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2’

shape_predictor_68_ 100%[===================>]  61.07M  39.0MB/s    in 1.6s    

2025-01-09 20:37:54 (39.0 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2’ saved [64040097/64040097]



In [18]:
import cv2
import dlib
import numpy as np
from IPython.display import display, HTML, Javascript
from google.colab.patches import cv2_imshow
from google.colab import output
from google.colab.output import eval_js
from base64 import b64decode
import threading
import ipywidgets as widgets
import time
import PIL
import io
import html

# Common Functions
def calculate_ear(eye):
    """Calculate the eye aspect ratio"""
    A = np.linalg.norm(eye[1] - eye[5])
    B = np.linalg.norm(eye[2] - eye[4])
    C = np.linalg.norm(eye[0] - eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

def process_frame(frame, face_detector, landmark_predictor):
    """Process a single frame for eye tracking"""
    if frame is None:
        return None

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)

    for face in faces:
        landmarks = landmark_predictor(gray, face)
        landmarks_points = np.array([(p.x, p.y) for p in landmarks.parts()])

        # Extract eye landmarks
        left_eye = landmarks_points[36:42]
        right_eye = landmarks_points[42:48]

        # Draw eye contours
        left_eye_hull = cv2.convexHull(left_eye)
        right_eye_hull = cv2.convexHull(right_eye)
        cv2.drawContours(frame, [left_eye_hull], -1, (0, 255, 0), 1)
        cv2.drawContours(frame, [right_eye_hull], -1, (0, 255, 0), 1)

        # Calculate EAR
        left_ear = calculate_ear(left_eye)
        right_ear = calculate_ear(right_eye)
        avg_ear = (left_ear + right_ear) / 2.0

        # Determine engagement
        gaze_direction = "Engaged" if avg_ear > 0.25 else "Distracted"

        # Display metrics
        cv2.putText(frame, f"EAR: {avg_ear:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
        cv2.putText(frame, f"Status: {gaze_direction}", (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

    return frame

# Method 1: Simple Frame-by-Frame Capture
def js_to_image(js_reply):
    """Convert JS response to OpenCV image"""
    image_bytes = b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def video_stream():
    js = Javascript('''
    async function setupWebcam() {
        const video = document.createElement('video');
        video.style.display = 'none';
        const stream = await navigator.mediaDevices.getUserMedia({video: true});
        video.srcObject = stream;
        await video.play();

        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);

        stream.getVideoTracks()[0].stop();
        return canvas.toDataURL('image/jpeg');
    }
    ''')
    display(js)
    return eval_js('setupWebcam()')

def run_simple_tracking():
    """Run eye tracking with simple frame-by-frame capture"""
    face_detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    try:
        while True:
            js_reply = video_stream()
            if js_reply:
                frame = js_to_image(js_reply)
                processed_frame = process_frame(frame, face_detector, landmark_predictor)
                cv2_imshow(processed_frame)
                output.clear()
            time.sleep(1)

    except KeyboardInterrupt:
        print("Stopped by user")
    except Exception as e:
        print(f"Error occurred: {str(e)}")

# Method 2: Continuous Display
def get_video_stream():
    """Create continuous video stream display"""
    js_code = """
    var video = document.createElement('video');
    var div = document.createElement('div');
    div.style.height = '400px';
    video.style.height = '400px';
    video.style.width = '600px';

    document.querySelector("#output-area").appendChild(div);
    div.appendChild(video);

    navigator.mediaDevices.getUserMedia({ video: true })
        .then(function(stream) {
            video.srcObject = stream;
            video.play();
        })
        .catch(function(err) {
            console.log("Error: " + err);
        });
    """
    display(Javascript(js_code))

def run_continuous_tracking():
    """Run eye tracking with continuous display"""
    face_detector = dlib.get_frontal_face_detector()
    landmark_predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    output_image = widgets.Image()
    display(output_image)

    js_code = """
    async function captureFrame() {
        const video = document.querySelector('video');
        const canvas = document.createElement('canvas');
        canvas.width = video.videoWidth;
        canvas.height = video.videoHeight;
        canvas.getContext('2d').drawImage(video, 0, 0);
        return canvas.toDataURL('image/jpeg');
    }
    """

    def update_frame():
        while True:
            try:
                frame_data = eval_js('captureFrame()')
                if frame_data:
                    frame_bytes = b64decode(frame_data.split(',')[1])
                    frame_array = np.frombuffer(frame_bytes, dtype=np.uint8)
                    frame = cv2.imdecode(frame_array, flags=1)

                    processed_frame = process_frame(frame, face_detector, landmark_predictor)
                    if processed_frame is not None:
                        _, buffer = cv2.imencode('.jpg', processed_frame)
                        output_image.value = buffer.tobytes()

                time.sleep(0.1)

            except Exception as e:
                print(f"Error: {str(e)}")
                break

    get_video_stream()
    display(Javascript(js_code))

    processing_thread = threading.Thread(target=update_frame)
    processing_thread.daemon = True
    processing_thread.start()

# Choose which method to run
def main(method='continuous'):
    if method == 'simple':
        run_simple_tracking()
    else:
        run_continuous_tracking()

In [ ]:
# For continuous display (recommended):
main('continuous')

# OR for simple frame-by-frame:
main('simple')

<IPython.core.display.Javascript object>